In [5]:
import sys
sys.path.append('..')

import os

from config import RUTA_UNIDAD_ONE_DRIVE
from config import RUTA_LOCAL_ONE_DRIVE

RUTA_COMPLETA = os.path.join(RUTA_UNIDAD_ONE_DRIVE, RUTA_LOCAL_ONE_DRIVE)

In [1]:
import pandas as pd

import logging
import logging.config
import pymysql
import paramiko
from paramiko import SSHClient
from sshtunnel import SSHTunnelForwarder

import ast

import openpyxl

In [10]:
RUTA_COMPLETA

'G:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON'

In [12]:
# ruta de carpera donde estan las llaves
ruta = open('../ruta_local.txt')
ruta = ruta.readlines()[0]

# cargar datos para conectar a AGROCITTCA
data = open(RUTA_COMPLETA + r'\_keys\agrocittca.data','r')
password = data.readlines()[0]
data.close()

# extraccion de los datos
datos_cnx = ast.literal_eval(password)

# lectura de llave ssh
mypkey = paramiko.RSAKey.from_private_key_file(RUTA_COMPLETA + '\_keys\llave_db_agrocittca', password='micontrasena')
# ssh variables
ssh_host = datos_cnx['ssh_host']
ssh_port = datos_cnx['ssh_port']
localhost = datos_cnx['localhost']
localport = datos_cnx['localport']
ssh_user = datos_cnx['ssh_user']
# database variables
sql_username = datos_cnx['sql_username']
sql_password = datos_cnx['sql_password']
sql_main_database = datos_cnx['sql_main_database']

In [13]:
# funcon para ejecutar una script SQL
def dbQuery(q):
    """ access the database over the SSH tunnel and execute the query """
    logger = logging.getLogger()
    logger.debug("Connecting to reporting server...", end="")
    with SSHTunnelForwarder(
        (ssh_host, ssh_port),
        ssh_username=ssh_user,
        ssh_pkey=mypkey,
        remote_bind_address=(localhost, 3306)) as tunnel:
        logger.debug("Connected")
        logger.debug("Connecting to database...")
        try:
            conn = pymysql.connect(
                host='127.0.0.1', 
                user=sql_username,
                passwd=sql_password,
                db=sql_main_database,
                port=tunnel.local_bind_port,
                cursorclass=pymysql.cursors.DictCursor)
            logger.debug("Connected")
            logger.debug("Executing query...")
            cursor = conn.cursor()
            cursor.execute(q)
            data = cursor.fetchall()
            logger.debug("Done")
        finally:
            conn.close()
    return data

In [14]:
def buscar_usuarios_dia(fecha_actual, ruta_xlxs):
    usuarios_act = dbQuery(f"select\
                                tok.created_at,\
                                usu.codigo,\
                                usu.name,\
                                usu.apellido_paterno,\
                                usu.apellido_materno,\
                                username\
                            from oauth_access_tokens tok\
                            inner join users usu on tok.user_id=usu.id\
                            where date(tok.created_at) = '{fecha_actual}'")
    lista_users_act = pd.DataFrame(usuarios_act)
    if len(lista_users_act) == 0:
        return
    lista_users_act = lista_users_act[~lista_users_act['name'].str.startswith('#')]
    wb = openpyxl.load_workbook(ruta_xlxs)
    ws = wb['Sheet1']
    for i in range(0, len(lista_users_act)):
        # Extrayendo la fila del DataFrame para agregar a Excel
        new_row = [
            lista_users_act.iloc[i, 0], 
            lista_users_act.iloc[i, 1], 
            lista_users_act.iloc[i, 2], 
            lista_users_act.iloc[i, 3], 
            lista_users_act.iloc[i, 4], 
            lista_users_act.iloc[i, 5]
        ]
        ws.append(new_row)
        wb.save(ruta_xlxs)
        wb.close()

In [19]:
ruta_xlxs = r'C:\Users\Usuario\Desktop\PROYECTOS\utea_scripts\agro_cittca\lista_usuarios_act.xlsx'

In [21]:
fecha_actual = '2024-05-12'
buscar_usuarios_dia(fecha_actual, ruta_xlxs)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Usuario\\Desktop\\PROYECTOS\\utea_scripts\\agro_cittca\\lista_usuarios_act.xlsx'